In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="abc")

# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def messages_for(website_content):
    system_prompt = """
    You are a helpful, funny assistant that summarizes the content of a website for dummies.
    """

    user_prompt = f"""
        Please summarize this website content:
        {website_content}
    """
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]


def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model="llama3.2:1b", messages=messages_for(website)
    )
    return response.choices[0].message.content


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


display_summary("https://en.wikipedia.org/wiki/Artificial_intelligence")
